<a href="https://colab.research.google.com/github/elaval/collab/blob/master/highered_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de librerías externas

In [0]:
!pip install numpy scipy pydash
!pip install resampy tensorflow six
!pip install tensorflowjs


    100% |████████████████████████████████| 92kB 5.5MB/s 
    100% |████████████████████████████████| 307kB 24.6MB/s 
    100% |████████████████████████████████| 13.8MB 1.9MB/s 
    100% |████████████████████████████████| 61kB 12.2MB/s 
    100% |████████████████████████████████| 51kB 22.5MB/s 
tensorflow 1.12.0 has requirement keras-applications>=1.0.6, but you'll have keras-applications 1.0.4 which is incompatible.
tensorflow 1.12.0 has requirement keras-preprocessing>=1.0.5, but you'll have keras-preprocessing 1.0.2 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
astropy 2.0.11 has requirement pytest<3.7,>=2.8, but you'll have pytest 3.10.1 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: numpy 1.14.6
    Un

In [0]:

import tensorflow as tf
from tensorflow import keras
import random

# Helper libraries
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import six

import pydash
from pydash import map_
from pydash import uniq
from pydash import sorted_uniq, uniq_by

import json



print(tf.__version__)

1.12.0


# Carga de datos de inacap - ingreso 2015

In [0]:
!curl -o inacap_2015.json https://raw.githubusercontent.com/elaval/datasets/master/desercion/inacap_2015.json


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.0M  100 18.0M    0     0  40.9M      0 --:--:-- --:--:-- --:--:-- 40.9M


In [0]:
with open('inacap_2015.json') as f:
    data = json.load(f)




# Feature extraction

In [0]:
features = ['ciudad_sede', 'codigo_unico', 'nem_cod_depe', 'nem_percentil', 'nivel_global', 'nomb_carrera', 'nomb_inst', 'origin', 'tipo_inst_1']

collection = {}
dict = {}
for f in features:
  collection[f] = sorted_uniq(map_(data, lambda x: x[f] if f in x else "N/A")) 
  dict[f] = {k: v for v, k in enumerate(collection[f])}

In [0]:
indices = {}  
tensor = {}

for f in features:
  indices[f] = map_(data, lambda x: dict[f][x[f] if f in x else "N/A"])
  tensor[f] = tf.one_hot(indices[f], len(collection[f]))
  

In [0]:
def tipoDesercion(item):
  
  if not item['potencialDesercion'] :
    tipo = 0
  elif item['potencialDesercion'] and item['fullDropout']:
    tipo = 1
  elif (item['potencialDesercion'] and item['cambioInstitucion'] ):
    tipo = 2
  else :
    tipo = 3
  return tipo

In [0]:
indice_desercion = map_(data, tipoDesercion )
tensorY = tf.one_hot(indice_desercion, 4)

# Combined features

In [0]:
combinedFeatures = ["ciudad_sede", "nomb_carrera"]

In [0]:
combinedTensorList = []
combinedDims = 0
for f in combinedFeatures:
  combinedTensorList.append(tensor[f])
  combinedDims = combinedDims + len(collection[f])
  
combinedTensor = tf.concat(combinedTensorList,1)

combinedModel = keras.Sequential([
    keras.layers.Dense(combinedDims, input_dim = combinedDims, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.softmax)
  ])
combinedModel.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [51]:
combinedModel.fit(combinedTensor, indice_desercion, epochs=5, steps_per_epoch= 100)


Epoch 1/5
100/100 [==============================] - 18s 181ms/step - loss: 1.0387 - acc: 0.5814
Epoch 2/5
100/100 [==============================] - 19s 189ms/step - loss: 1.0372 - acc: 0.5817
Epoch 3/5
100/100 [==============================] - 19s 188ms/step - loss: 1.0362 - acc: 0.5818
Epoch 4/5
100/100 [==============================] - 19s 189ms/step - loss: 1.0354 - acc: 0.5819
Epoch 5/5
100/100 [==============================] - 19s 188ms/step - loss: 1.0349 - acc: 0.5819


## Save combined model

In [64]:
combinedModelPath = ""
for i, f  in enumerate(combinedFeatures):
  combinedModelPath =  combinedModelPath + ("_" if i > 0 else "") + f 
combinedModel.save('out/model_%s.h5' % combinedModelPath)
!tensorflowjs_converter --input_format=keras ./out/model_{combinedModelPath}.h5 ./out/model_{combinedModelPath}_tfjs


Using TensorFlow backend.


# Carga de modelos

In [17]:
!mkdir out
model = {}
for f in features:
  !curl -o out/model__{f}.h5 https://raw.githubusercontent.com/elaval/datasets/master/desercion_inacap/model__{f}.h5
  
  modelPath = "out/model__%s.h5" % f
  model[f] = tf.keras.models.load_model(
    modelPath,
    custom_objects=None,
    compile=True
  )
  model[f].compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

mkdir: cannot create directory ‘out’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14416  100 14416    0     0   281k      0 --:--:-- --:--:-- --:--:--  281k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4514k  100 4514k    0     0  20.1M      0 --:--:-- --:--:-- --:--:-- 20.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14416  100 14416    0     0    98k      0 --:--:-- --:--:-- --:--:--   99k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14416  100 14416    0     0    99k      0 --:--:-- --:--:-- --:--:--   99k
  

# Definición de modelos

## Create models from scratch
When models have not been loaded

In [0]:
model = {}
for f in features:
  dims = len(collection[f])
  model[f] = keras.Sequential([
    keras.layers.Dense(dims, input_dim = dims, activation=tf.nn.relu),
    keras.layers.Dense(4, activation=tf.nn.softmax)
  ])
  model[f].compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  

## Train models

In [0]:
num_epochs = 1

In [21]:
for f in features:
  model[f].fit(tensor[f], indice_desercion, epochs=num_epochs, steps_per_epoch= 100)

Epoch 1/1
100/100 [==============================] - 3s 29ms/step - loss: 1.1594 - acc: 0.5466
Epoch 1/1
100/100 [==============================] - 404s 4s/step - loss: 0.9905 - acc: 0.5960
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 1.1612 - acc: 0.5466
Epoch 1/1
100/100 [==============================] - 4s 41ms/step - loss: 1.1485 - acc: 0.5466
Epoch 1/1
100/100 [==============================] - 3s 34ms/step - loss: 1.1653 - acc: 0.5486
Epoch 1/1
100/100 [==============================] - 18s 177ms/step - loss: 1.0674 - acc: 0.5747
Epoch 1/1
100/100 [==============================] - 2s 22ms/step - loss: 1.1456 - acc: 0.5466
Epoch 1/1
100/100 [==============================] - 4s 37ms/step - loss: 1.1460 - acc: 0.5467
Epoch 1/1
100/100 [==============================] - 2s 23ms/step - loss: 1.1462 - acc: 0.5466


# Almacenamiento de modelos y datos

In [33]:
for f in features:
  print("Feature: %s" % f)
  # Save collections
  with open('out/%s.txt' % f, 'w') as file:
    file.write("%s\n" % f)
    for item in collection[f]:
      if isinstance(item, six.string_types):
        file.write("%s\n" % item.encode('utf-8'))
      else:
        file.write("%d\n" % item)
  model[f].save('out/model_%s.h5' % f)
  !tensorflowjs_converter --input_format=keras ./out/model_{f}.h5 ./out/model_{f}_tfjs


Feature: ciudad_sede
Using TensorFlow backend.
Feature: codigo_unico
Using TensorFlow backend.
Feature: nem_cod_depe
Using TensorFlow backend.
Feature: nem_percentil
Using TensorFlow backend.
Feature: nivel_global
Using TensorFlow backend.
Feature: nomb_carrera
Using TensorFlow backend.
Feature: nomb_inst
Using TensorFlow backend.
Feature: origin
Using TensorFlow backend.
Feature: tipo_inst_1
Using TensorFlow backend.


In [34]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r out "drive/My Drive/_tensorflow"

